In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.time import Time

%matplotlib inline

In [2]:
DATA_DIR = "../data"

In [3]:
observations = pd.read_csv(
    os.path.join(DATA_DIR, "observations.txt"), 
    index_col=False, 
    sep=" "
)

all_linkages = pd.read_csv(
    os.path.join(DATA_DIR, "all_linkages.txt"), 
    index_col=False, 
    sep=" "
)

linkage_members = pd.read_csv(
    os.path.join(DATA_DIR, "linkage_members.txt"), 
    index_col=False, 
    sep=" "
)

In [4]:
def writeADESHeader(observatory_code, 
                    submitter, 
                    telescope_design,
                    telescope_aperture,
                    telescope_detector,
                    observers,
                    measurers,
                    observatory_name=None,
                    submitter_institution=None,
                    telescope_name=None,
                    comment=None):
    
    # Start header with version number
    header = [
        "# version=2017",
    ]
    
    # Add observatory [required]
    header += ["# observatory"]
    header += ["! mpcCode {}".format(observatory_code)]
    if observatory_name is not None:
        header += ["! name {}".format(observatory_name)]
    
    # Add submitter [required]
    header += ["# submitter"]
    header += ["! name {}".format(submitter)]

    if submitter_institution is not None:
        header += ["! institution {}".format(submitter_institution)]

    # Add telescope details [required]
    header += ["# telescope"]
    if telescope_name is not None:
        header += ["! name {}".format(telescope_name)]
    header += ["! aperture 8.4"]
    header += ["! design {}".format(telescope_design)]
    header += ["! detector {}".format("CCD")]

    # Add observer details
    header += ["# observers"]
    if type(observers) is not list:
        err = (
            "observers should be a list of strings."
        )
        raise ValueError(err)
    for name in observers:
        header += ["! name {}".format(name)]

    # Add measurer details
    header += ["# measurers"]
    if type(measurers) is not list:
        err = (
            "measurers should be a list of strings."
        )
        raise ValueError(err)
    for name in measurers:
        header += ["! name {}".format(name)]

    # Add comment
    if comment is not None:
        header += ["# comment"]
        header += ["! line {}".format(comment)]
        
    header = [i + "\n" for i in header]
    return header

def writeToADES(observations, 
                file_out,
                observatory_code, 
                submitter, 
                telescope_design,
                telescope_aperture,
                telescope_detector,
                observers,
                measurers,
                mjd_scale="utc",
                **header_kwargs):
    
    header = writeADESHeader(
        observatory_code, 
        submitter, 
        telescope_design,
        telescope_aperture,
        telescope_detector,
        observers,
        measurers,
        **header_kwargs
    )
    
    # Format columns from observations into PSV format
    ades = {}
    
    id_present = False
    if "permID" in observations.columns.values:
        ades["permID"] = observations["permID"].values
        id_present = True
    if "provID" in observations.columns.values:
        ades["provID"] = observations["permID"].values
        id_present = True
        
    if "trkSub" in observations.columns.values:
        ades["trkSub"] = observations["trkSub"].values
        id_present = True
    
    if not id_present:
        err = (
            "At least one of permID, provID, or trkSub should\n"
            "be present in observations."
        )
        raise ValueError(err)
    
    observation_times = Time(
        observations["mjd_utc"].values,
        format="mjd",
        scale=mjd_scale
    )
    ades["obsTime"] = np.array([i + "Z" for i in observation_times.utc.isot])
    ades["ra"] = observations["ra"].values 
    ades["dec"] = observations["dec"].values

    if "rmsRA" in observations.columns.values:
        ades["rmsRA"] = observations["rmsRA"].values
    if "rmsDec" in observations.columns.values:
        ades["rmsDec"] = observations["rmsDec"].values
    
    ades["mag"] = observations["mag"].values
    if "rmsMag" in observations.columns.values:
        ades["rmsMag"] = observations["rmsMag"].values
    ades["band"] = observations["band"].values
    ades["stn"] = observations["stn"].values
    ades["mode"] = observations["mode"].values
    ades["astCat"] = observations["astCat"].values

    # Create dataframe with formated data entries
    ades = pd.DataFrame(ades)
    col_header = "|".join(ades.columns) + "\n"
   
    with open(file_out, "w") as f:
        f.write("".join(header))
        f.write(col_header)
        
    ades.to_csv(file_out, sep="|", header=False, index=False, mode="a")
    return 

In [5]:
linkage_members[linkage_members["trkSub"].isin(["t0000000", "t0000001"])]["obs_id"].values

array(['obs00013782', 'obs00014297', 'obs00243208', 'obs00243490',
       'obs00243694', 'obs00244204', 'obs00244574', 'obs00245241',
       'obs00247036', 'obs00248712', 'obs00287068', 'obs00287578',
       'obs00074394', 'obs00074412', 'obs00074416', 'obs00074435',
       'obs00244082', 'obs00245166', 'obs00245635', 'obs00245656',
       'obs00246352', 'obs00247012', 'obs00248680', 'obs00286981',
       'obs00287150', 'obs00287441', 'obs00287710', 'obs00288929',
       'obs00290270'], dtype=object)

In [6]:
linkage_members

,trkSub,obs_id
0,t0000000,obs00013782
1,t0000000,obs00014297
2,t0000000,obs00243208
3,t0000000,obs00243490
4,t0000000,obs00243694
...,...,...
2791573,t0300691,obs03163025
2791574,t0300691,obs03696719
2791575,t0300691,obs03740561
2791576,t0300691,obs05946913


In [7]:
observations_linked = linkage_members.merge(observations, on="obs_id")

In [8]:
observations_linked.head()

,trkSub,obs_id,obj_id,mjd_utc,ra,dec,rmsRA,rmsDec,band,mag,rmsMag,stn,mode,astCat,night
0,t0000000,obs00013782,S00000Bba,59854.040282,321.235889,-56.059465,0.017320,0.031021,y,20.664,0.055757,I11,CCD,Gaia2,59854
1,t0000000,obs00014297,S00000Bba,59854.055238,321.240279,-56.046750,0.023409,0.041912,y,20.665,0.065089,I11,CCD,Gaia2,59854
2,t0000000,obs00243208,S00000Bba,59856.008216,321.868249,-54.403628,0.009061,0.015567,z,20.725,0.027443,I11,CCD,Gaia2,59856
3,t0000000,obs00243490,S00000Bba,59856.024389,321.872979,-54.390224,0.006883,0.011821,i,20.671,0.016636,I11,CCD,Gaia2,59856
4,t0000000,obs00243694,S00000Bba,59856.039706,321.877452,-54.377512,0.006982,0.011987,i,20.672,0.017060,I11,CCD,Gaia2,59856


In [9]:
observatory_code = "I11"
observatory_name = "Vera C. Rubin Observatory"
telescope_aperture = "8.4"
telescope_design = "Reflector"
telescope_detector = "CCD"
submitter = "D. iRAC"
observers = ["D. iRAC"]
measurers = ["D. iRAC"]

writeToADES(
    observations_linked[observations_linked["trkSub"].isin(["t0000000", "t0000001"])],
    "2unknown.psv",
    observatory_code, 
    submitter, 
    telescope_design,
    telescope_aperture,
    telescope_detector,
    observers,
    measurers,
    comment="MPC-Rubin Observatory Integration Tests"
)

In [10]:
! curl https://minorplanetcenter.net/submit_psv_test -F "ack=ADES Code Testing" -F "ac2=moeyensj@uw.edu" -F "obj_type=Unclassified" -F "source=<2unknown.psv"

Submission format valid.
